# Question Answering using the Falcon-7b Model (Pipeline from HuggingFace)

In this section, we utilize the `tiiuae/falcon-7b` model to perform question answering based on a given context. The aim is to extract answers from a context about the Bash shell.

## Code Overview:

- **Model & Tokenizer Initialization**: We initialize the model and load the associated tokenizer.

```python
  model = "tiiuae/falcon-7b"
  tokenizer = AutoTokenizer.from_pretrained(model) 
```

- **Pipeline Configuration**: The text-generation pipeline is established with defined parameters.

`torch_dtype: Indicates the datatype as bfloat16 since the models were trained using this type.
trust_remote_code: Determines if the pipeline should trust and execute remote code from the model.
device_map: Sets the computing device (CPU/GPU) for the execution.

```python
pipeline = transformers.pipeline(
    "text-generation",
    ...
)
```

- **Question Answering**: A context is provided to the pipeline, followed by a question, prompting the model to generate an appropriate answer.

```python
max_length: Maximum length for the output.
do_sample: Enables the use of sampling in text generation.
top_k: Constrains the set of possible next words during sampling.
eos_token_id: Denotes the token ID for the end-of-sequence.

sequences = pipeline(
   "Based on the following context, answer the question. ...",
    ...
)
```

- **Output Display**: The generated answer based on the context and question is printed.

```python
for seq in sequences:
    print(f"Result: {seq['generated_text']}")
```    
    

In [1]:
from transformers import AutoTokenizer
import transformers
import torch

import warnings
warnings.filterwarnings('ignore')



In [7]:
def print_result(sequences):
    for idx, entry in enumerate(sequences):
        text = entry['generated_text']

        context_start = text.find("Context:")
        context_end = text.find("Question:")
        
        
        answer_pos = text.find("Answer:", context_end)
        newline_pos = text.find("\n", context_end)
        
        if answer_pos == -1:
            answer_pos = float('inf')
        
        
        if newline_pos == -1:
            newline_pos = float('inf')
        
        
        question_end = min(answer_pos, newline_pos)
        

        prompt = text[:context_start].strip()
        context = text[context_start:context_end].replace("Context:", "").strip()
        question = text[context_end:question_end].replace("Question:", "").strip()
        answer = text[question_end:].strip()

        print(f"Result {idx + 1} :")
        print("\nPrompt :", prompt)
        print("Context :", context)
        print("Question :", question)
        print("\n", answer)
        print("\n" + '-' * 50)  # separator line

In [3]:
model = "tiiuae/falcon-7b"


In [4]:
tokenizer = AutoTokenizer.from_pretrained(model)


- The models were trained using the bfloat16 datatype.
- This requires a recent version of CUDA and works best on modern cards. 
- Try to run inference using float16, but keep in mind that the models were evaluated using bfloat16.



In [5]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16, #The models were trained using the bfloat16 datatype
    trust_remote_code=True, #controls whether the pipeline should trust and execute remote code provided by the model 
    device_map="auto",
)

2023-09-29 12:41:42.571061: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-29 12:41:43.718242: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/r.nair/.local/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA SETUP: CUDA runtime path found: /shared/centos7/cuda/11.8/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/r.nair/.local/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


A Jupyter Widget

A Jupyter Widget

In [23]:
sequences = pipeline(
    "Based on the following context, answer the question. Context: Overview# Bash (Bourne Again SHell) is a popular shell and command-line interface. Specifically, a shell is an interface between the user and the underlying operating system, allowing users to interact with the system and perform tasks. Bash provides a range of features for running commands, managing files, navigating systems, and performing other tasks. Bash commands perform various tasks within the shell environment. Commands span basic functionalities (ls, cd, cp, mv, and rm) to more advanced ones(e.g., grep and awk). We cover these commands and more in this tutorial. Bash can also be used in scripts, allowing users to automate tasks and perform more complex operations via loops, conditional logic, and defining functions. Question: What are some of the commands in bash?",
    max_length=600,    # the maximum length of the output sequences.
    do_sample=True,    #whether or not to use sampling in generating text. 
    top_k=40,          #used during sampling to restrict the set of words considered for the next word in the sequence.
    num_return_sequences=2,     #Specifies the number of independently generated sequences to return for a given input.
    eos_token_id=tokenizer.eos_token_id,     #specifies the token ID of the end-of-sequence (EOS) token.
)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



- max_length    # the maximum length of the output sequences.

- do_sample=True     #whether or not to use sampling in generating text. 
    When do_sample is True, the model will randomly pick the next word according to its predicted word probabilities, 
    which leads to more diverse and creative outputs. 
    This contrasts with just choosing the word with the highest predicted probability (i.e., deterministic approach) 
    which could lead to more predictable text.

- top_k=20: The model will consider only the top 20 words (based on predicted probabilities) and sample the next word 
    from this subset. This can prevent very rare words from being selected and can lead to more coherent text.

- num_return_sequences=2:

- Purpose:The model will generate two different sequences for the given input. 
    This can be useful to get multiple variations of the generated text for a single prompt.

- eos_token_id=tokenizer.eos_token_id: 
    Value: The value derived from the tokenizer represents the ID for the end-of-sequence token.
    When the model generates this token, it will treat it as the end of the sequence and stop generating further tokens. 
    This can be useful if you want the model to consider stopping before reaching the max_length based on the context or 
    the internal structure of the text it is generating.





In [24]:
print_result(sequences)

Result 1 :

Prompt : Based on the following context, answer the question.
Context : Overview# Bash (Bourne Again SHell) is a popular shell and command-line interface. Specifically, a shell is an interface between the user and the underlying operating system, allowing users to interact with the system and perform tasks. Bash provides a range of features for running commands, managing files, navigating systems, and performing other tasks. Bash commands perform various tasks within the shell environment. Commands span basic functionalities (ls, cd, cp, mv, and rm) to more advanced ones(e.g., grep and awk). We cover these commands and more in this tutorial. Bash can also be used in scripts, allowing users to automate tasks and perform more complex operations via loops, conditional logic, and defining functions.
Question : What are some of the commands in bash? Answers: The following are some of the commands in bash: cd(Change Directory )-This allows users to navigate their filesystem by cd

#### Falcon-7b-instruct



- Technology Innovation Institute (TII) has made available instruct versions of the Falcon model, Falcon-7B-Instruct and Falcon-40B-Instruct. These experimental variants have been finetuned on instructions and conversational data; they thus lend better to popular assistant-style tasks.

In [1]:
from transformers import AutoTokenizer
import transformers
import torch

import warnings
warnings.filterwarnings('ignore')



In [2]:
model2 = "tiiuae/falcon-7b-instruct"



In [3]:
tokenizer = AutoTokenizer.from_pretrained(model2)

A Jupyter Widget

In [4]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model2,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16, #The models were trained using the bfloat16 datatype
    trust_remote_code=True, #controls whether the pipeline should trust and execute remote code provided by the model 
    device_map="auto",
)

2023-09-29 12:52:05.455154: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-29 12:52:06.558795: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/r.nair/.local/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA SETUP: CUDA runtime path found: /shared/centos7/cuda/11.8/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/r.nair/.local/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


A Jupyter Widget

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.




A Jupyter Widget

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

In [5]:



sequences = pipeline(
    "Based on the following context, answer the question. Context: Overview# Bash (Bourne Again SHell) is a popular shell and command-line interface. Specifically, a shell is an interface between the user and the underlying operating system, allowing users to interact with the system and perform tasks. Bash provides a range of features for running commands, managing files, navigating systems, and performing other tasks. Bash commands perform various tasks within the shell environment. Commands span basic functionalities (ls, cd, cp, mv, and rm) to more advanced ones(e.g., grep and awk). We cover these commands and more in this tutorial. Bash can also be used in scripts, allowing users to automate tasks and perform more complex operations via loops, conditional logic, and defining functions. Question: What are some of the commands in bash?",
    max_length=500,         # the maximum length of the output sequences.
    do_sample=True,         #whether or not to use sampling in generating text. 
    top_k=20,                 #used during sampling to restrict the set of words considered for the next word in the sequence.
    num_return_sequences=2,     #Specifies the number of independently generated sequences to return for a given input.
    eos_token_id=tokenizer.eos_token_id,       #specifies the token ID of the end-of-sequence (EOS) token.
)

    
  

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


In [6]:
sequences

[{'generated_text': 'Based on the following context, answer the question. Context: Overview# Bash (Bourne Again SHell) is a popular shell and command-line interface. Specifically, a shell is an interface between the user and the underlying operating system, allowing users to interact with the system and perform tasks. Bash provides a range of features for running commands, managing files, navigating systems, and performing other tasks. Bash commands perform various tasks within the shell environment. Commands span basic functionalities (ls, cd, cp, mv, and rm) to more advanced ones(e.g., grep and awk). We cover these commands and more in this tutorial. Bash can also be used in scripts, allowing users to automate tasks and perform more complex operations via loops, conditional logic, and defining functions. Question: What are some of the commands in bash?\nSome of the commands in Bash include "ls" to list files and folders, "cd" to change the directory you\'re currently in, "cp" to copy

In [8]:
print_result(sequences)

Result 1 :

Prompt : Based on the following context, answer the question.
Context : Overview# Bash (Bourne Again SHell) is a popular shell and command-line interface. Specifically, a shell is an interface between the user and the underlying operating system, allowing users to interact with the system and perform tasks. Bash provides a range of features for running commands, managing files, navigating systems, and performing other tasks. Bash commands perform various tasks within the shell environment. Commands span basic functionalities (ls, cd, cp, mv, and rm) to more advanced ones(e.g., grep and awk). We cover these commands and more in this tutorial. Bash can also be used in scripts, allowing users to automate tasks and perform more complex operations via loops, conditional logic, and defining functions.
Question : What are some of the commands in bash?

 Some of the commands in Bash include "ls" to list files and folders, "cd" to change the directory you're currently in, "cp" to co